# National Income Inequality Diagonstic

## Notebook - 02 - Data Cleaning and reshaping

**Objective:**
Convert raw World Bank Gini Index data into a clean, long-format,
analysis-ready CSV without changing the file format.

In [16]:
import numpy as np
import pandas as pd

In [17]:
gini_raw = pd.read_csv("../data/raw/world_bank_gini.csv", skiprows=4)

In [18]:
#keep only required columns
cols_to_keep = ["Country Name","Country Code"] + list(gini_raw.columns[4:])
gini_reduced = gini_raw[cols_to_keep]

In [23]:
#Rename Columns
gini_reduced = gini_reduced.rename(columns={
    "Country Name": "country",
    "Country Code": "country_code"
})

In [24]:
# Wide -  longe reshape 
gini_long = gini_reduced.melt(
    id_vars = ["country", "country_code"],
    var_name = "year",
    value_name="gini"
)
gini_long

,country,country_code,year,gini
0,Aruba,ABW,1960,NaN
1,Africa Eastern and Southern,AFE,1960,NaN
2,Afghanistan,AFG,1960,NaN
3,Africa Western and Central,AFW,1960,NaN
4,Angola,AGO,1960,NaN
...,...,...,...,...
17551,Kosovo,XKX,Unnamed: 69,NaN
17552,"Yemen, Rep.",YEM,Unnamed: 69,NaN
17553,South Africa,ZAF,Unnamed: 69,NaN
17554,Zambia,ZMB,Unnamed: 69,NaN


In [27]:
# Data Types Fix
gini_long["year"] = pd.to_numeric(gini_long["year"],errors = "coerce")
gini_long["gini"] = pd.to_numeric(gini_long["gini"],errors = "coerce")

In [29]:
# Drop Empty Rows
gini_long_clean = gini_long.dropna(subset=["gini"])
gini_long_clean

,country,country_code,year,gini
1049,United States,USA,1963.0,36.7
1315,United States,USA,1964.0,37.4
1581,United States,USA,1965.0,36.8
1847,United States,USA,1966.0,37.1
2113,United States,USA,1967.0,36.9
...,...,...,...,...
17210,Peru,PER,2024.0,40.1
17219,Paraguay,PRY,2024.0,44.2
17258,Tajikistan,TJK,2024.0,36.1
17274,Uruguay,URY,2024.0,40.0


In [31]:
#Save Processed CSV
gini_long_clean.to_csv("../data/processed/world_bank_gini_cleaned.csv", index=False)

In [34]:
# Load Country Metadata
country_meta = pd.read_csv("../data/raw/Metadata_Country.csv")

In [35]:
# Select Relevant Metadata Columns
country_meta_reduced = country_meta[
    ["Country Code","Region","IncomeGroup"]
].rename(columns={
    "Country Code": "country_code",
    "IncomeGroup":"income_group"
})

In [36]:
# Merge Metadata
gini_enriched = gini_long_clean.merge(
    country_meta_reduced,
    on="country_code",
    how="left"
)

In [43]:
# Save Enriched CSV
gini_enriched.to_csv("../data/processed/metadata_country_enriched.csv", index=False)
gini_enriched.head(),gini_enriched.shape


(         country country_code    year  gini         Region income_group
 0  United States          USA  1963.0  36.7  North America  High income
 1  United States          USA  1964.0  37.4  North America  High income
 2  United States          USA  1965.0  36.8  North America  High income
 3  United States          USA  1966.0  37.1  North America  High income
 4  United States          USA  1967.0  36.9  North America  High income,
 (2402, 6))